In [1]:
import CodaPCA
import CodaRegressmb as CodaRegress
import numpy as np
from runpca import read_csv
import os
import sklearn
import torch
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
#change module for newer sklearn versions
from sklearn.model_selection  import cross_val_score
from sklearn.model_selection  import KFold
import matplotlib.pyplot as plt
import pandas as pd
import PCARegress
import itertools

In [2]:
#given features, targerts, folds and params, return kfold scores 

def coda_val(features, targets, n_components, folds, nn_shape, learn_rate, lam,  n_epochs):
    targets = targets.reshape(-1,1)
    kfold_scores = []
    for train, test in folds:        
        Y_train = targets[train]
        X_train = features[train]
        
        Y_test = targets[test]
        X_test = features[test]
        
        #compute test score based on best parameter
        
        model = CodaRegress.CoDA_Regress(features.shape[1], n_components, nn_shape[0], nn_shape[1])

        model.fit(torch.FloatTensor(X_train),  torch.FloatTensor(Y_train),
                  lam, lr=learn_rate, train_size = int(len(X_train)*(3/4)), epochs=n_epochs)

        y_pred = model.predict(torch.FloatTensor(X_test))
        kfold_scores.append(sklearn.metrics.mean_squared_error(Y_test,y_pred.detach().numpy()))
                
    return kfold_scores

In [19]:
def test_aitchinson_data(data, co_feature_indices, target_index, 
                   param_list, ind, other_feature_indices = []):
    
    headers = data[1]
    
    features = data[0][:,co_feature_indices]
    targets = data[0][:,target_index]
    
    #normalise the compositional features. TODO anything extra to deal with non compositional features?
    features = np.array([feat/sum(feat) for feat in features])

    n_components=2  #len(co_feature_indices)-2
    
    splits = 4
        
    #split data 
    kf = KFold(splits)
        
    folds = [i for i in kf.split(features)]
    
    #with folds fixed for fair comparison, loop through scores using all combination of parameters
    
    param_file = open("paramresults" + str(ind) + ".txt","w") 
    param_file.write("Results from hyperparameter grid search:\n\n") 
        
        
        
    for params in param_list:
        nn_shape = params[0]
        lr = params[1]
        lam = params[2]
        n_epochs = params[3]
        
        score = coda_val(features, targets, 2, folds, nn_shape, lr, lam, n_epochs )

        param_file.write("nn {}, lr {}, lam {}, epochs {}, num_samples {}, num_features {} \nScore: {} \nMean_score: {} \n\n".format(nn_shape, lr, lam, n_epochs, len(features), len(co_feature_indices), score, np.mean(score)))
    
    param_file.close()
    
    return 


In [21]:


nn_vals = [[[5,], [2,]], [[10,],[5,]], [[10,5], [3,5]], [[50,], [20,]], [[50,20], [10,20]]]
lr_vals = [1e-1,1e-2,5e-2,1e-3]
lam_vals = [1e-16,1e-12,5e-12,1e-14,1e-10,1e-5,1e-4,0.001,0.1,1,10,100]
epoch_vals = [100, 1000, 5000]

param_list = list(itertools.product(*[nn_vals, lr_vals, lam_vals, epoch_vals]))


score_dict = {} 

#TODO r_files isn't always consistent

data_21_scores = test_aitchinson_data(read_csv("Aitchinson/Data 21. Permeabilities of bayesite for 21 mixtures of fibres and bonding pressures..csv", normalize=False), co_feature_indices=[0,1,2,3], target_index=4, param_list = param_list, ind = 21)
score_dict['21'] = data_21_scores

data_21_scores2 =test_aitchinson_data(read_csv("Aitchinson/Data 21. Permeabilities of bayesite for 21 mixtures of fibres and bonding pressures..csv", normalize=False), co_feature_indices=[0,1,2,3], target_index=5, param_list = param_list, ind = 212)
score_dict['21,2'] = data_21_scores2

data_34_scores = test_aitchinson_data(read_csv("Aitchinson/Data 34. Foraminiferal compositions at 30 different depths.csv", normalize=False), co_feature_indices=[0,1,2,3], target_index=4, param_list = param_list, ind = 34)
score_dict['34'] = data_34_scores

data_3_scores = test_aitchinson_data(read_csv("Aitchinson/Data 3. Compositions and depths of 25 specimens of boxite (Percentages by weight).csv", normalize=False), co_feature_indices=[0,1,2,3,4], target_index=5, param_list = param_list, ind = 3)
score_dict['3'] = data_3_scores

data_39_scores = test_aitchinson_data(read_csv("Aitchinson/Data 39. Microhardness of 18 glass specimens and their (Ge, Sb, Se) compositions.csv", normalize=False), co_feature_indices=[0,1,2], target_index=3, param_list = param_list, ind = 39)
score_dict['39'] = data_39_scores

data_4_scores = test_aitchinson_data(read_csv("Aitchinson/Data 4. Compositions, depths and porosities of 25 specimens of coxite (Percentages by weight).csv", normalize=False), co_feature_indices=[0,1,2,3,4], target_index=5, param_list = param_list, ind = 4)
score_dict['4'] = data_4_scores

data_4_scores2 = test_aitchinson_data(read_csv("Aitchinson/Data 4. Compositions, depths and porosities of 25 specimens of coxite (Percentages by weight).csv", normalize=False), co_feature_indices=[0,1,2,3,4], target_index=6, param_list = param_list, ind = 42)
score_dict['4,2'] = data_4_scores2

data_5_scores = test_aitchinson_data(read_csv("Aitchinson/Data 5. Sand, silt, clay compositions of 39 sediment samples at different water depths in an Arctic lake.csv", normalize=False), co_feature_indices=[0,1,2], target_index=3, param_list = param_list, ind = 5)
score_dict['5'] = data_5_scores

data_18_scores = test_aitchinson_data(read_csv("Aitchinson/Data 18. Compositions and total pebble counts of 92 glacial tills.csv", normalize=False), co_feature_indices=[0,1,2], target_index=3, param_list = param_list, ind = 18)
score_dict['18'] = data_18_scores


loading Aitchinson/Data 21. Permeabilities of bayesite for 21 mixtures of fibres and bonding pressures..csv...
21 samples 6 features
sparsity: 0.0%
epoch 1000, loss 2.1618804931640625
epoch 1000, loss 4.237913608551025
epoch 2000, loss 4.230597019195557
epoch 1000, loss 3.7385010719299316
epoch 2000, loss 7.631176413269714e-05
epoch 3000, loss 0.00686924671754241
epoch 4000, loss 0.11667885631322861
epoch 1000, loss 0.805679440498352
epoch 1000, loss 0.3039858639240265
epoch 2000, loss 0.2866445481777191
epoch 3000, loss 0.2860166132450104
epoch 4000, loss 0.2564489543437958
epoch 1000, loss 4.209451675415039
epoch 2000, loss 0.413632869720459
epoch 3000, loss 0.3710266351699829
epoch 4000, loss 0.3492278456687927
epoch 1000, loss 2.8517353534698486
epoch 1000, loss 0.7767089009284973
epoch 1000, loss 2.163102626800537
epoch 2000, loss 0.1385963261127472
epoch 1000, loss 4.197796821594238
epoch 1000, loss 3.362943410873413
epoch 1000, loss 0.7947605848312378
epoch 1000, loss 0.79016143

epoch 1000, loss 4.354454040527344
epoch 1000, loss 4.498224258422852
epoch 2000, loss 2.6331560611724854
epoch 1000, loss 0.8201797008514404
epoch 1000, loss 4.465106964111328
epoch 2000, loss 0.6304925084114075
epoch 3000, loss 0.5265361666679382
epoch 4000, loss 0.27239006757736206
epoch 1000, loss 4.371114730834961
epoch 2000, loss 2.6560375690460205
epoch 1000, loss 5.053847312927246
epoch 1000, loss 8.633708953857422
epoch 1000, loss 41.83705139160156
epoch 1000, loss 43.64656066894531
epoch 1000, loss 410.38006591796875
epoch 1000, loss 38.21180725097656
epoch 2000, loss 16.742076873779297
epoch 3000, loss 7.84943151473999
epoch 4000, loss 5.341061592102051
epoch 1000, loss 3.5120768547058105
epoch 2000, loss 3.1138458251953125
epoch 3000, loss 2.834446430206299
epoch 4000, loss 2.5304555892944336
epoch 1000, loss 5.2418107986450195
epoch 2000, loss 4.84517240524292
epoch 3000, loss 4.6267924308776855
epoch 4000, loss 4.519268989562988
epoch 1000, loss 4.96569299697876
epoch 200

epoch 1000, loss 8.675142288208008
epoch 1000, loss 8.014729499816895
epoch 2000, loss 4.560628414154053
epoch 3000, loss 4.445459842681885
epoch 1000, loss 44.45699691772461
epoch 1000, loss 47.221221923828125
epoch 1000, loss 414.0630798339844
epoch 1000, loss 0.9157896041870117
epoch 2000, loss 0.8393020033836365
epoch 1000, loss 2.204773426055908
epoch 2000, loss 2.0227125787641853e-06
epoch 3000, loss 2.0938721676661e-12
epoch 4000, loss 4.9007184088623035e-08
epoch 1000, loss 3.776618719100952
epoch 1000, loss 4.493537902832031
epoch 2000, loss 0.01849057525396347
epoch 3000, loss 0.0001943440001923591
epoch 4000, loss 0.00017258262960240245
epoch 1000, loss 0.9509462118148804
epoch 2000, loss 0.7012145519256592
epoch 3000, loss 0.6920878291130066
epoch 1000, loss 2.1904144287109375
epoch 2000, loss 9.686329576652497e-05
epoch 3000, loss 6.611666075739819e-11
epoch 4000, loss 1.1179585035492323e-10
epoch 1000, loss 4.467471122741699
epoch 2000, loss 1.5833587646484375
epoch 1000,

epoch 1000, loss 24.794963836669922
epoch 2000, loss 10.331764221191406
epoch 3000, loss 5.809220790863037
epoch 4000, loss 3.679547071456909
epoch 1000, loss 3.1605184078216553
epoch 2000, loss 2.935891628265381
epoch 3000, loss 2.607743978500366
epoch 4000, loss 2.258931875228882
epoch 1000, loss 4.973954200744629
epoch 2000, loss 4.606884956359863
epoch 3000, loss 4.504845142364502
epoch 4000, loss 4.490609169006348
epoch 1000, loss 5.0310444831848145
epoch 2000, loss 4.848023891448975
epoch 3000, loss 4.700297832489014
epoch 4000, loss 4.634085178375244
epoch 1000, loss 29.315919876098633
epoch 2000, loss 12.405346870422363
epoch 3000, loss 6.392196178436279
epoch 4000, loss 2.4967896938323975
epoch 1000, loss 3.194653272628784
epoch 2000, loss 2.794860601425171
epoch 3000, loss 2.2972960472106934
epoch 4000, loss 2.216947078704834
epoch 1000, loss 5.192791938781738
epoch 2000, loss 4.800497055053711
epoch 3000, loss 4.5782470703125
epoch 4000, loss 4.510427474975586
epoch 1000, lo

epoch 1000, loss 0.4273403286933899
epoch 2000, loss 0.00043512776028364897
epoch 1000, loss 5.184831619262695
epoch 2000, loss 1.5766327381134033
epoch 1000, loss 3.0976357460021973
epoch 1000, loss 0.8229460120201111
epoch 2000, loss 0.0043623242527246475
epoch 3000, loss 0.00410127080976963
epoch 4000, loss 0.00404534162953496
epoch 1000, loss 0.829217791557312
epoch 2000, loss 0.004390254616737366
epoch 1000, loss 4.4801411628723145
epoch 2000, loss 4.084098815917969
epoch 3000, loss 0.0044967494904994965
epoch 4000, loss 0.007608234882354736
epoch 1000, loss 4.99834680557251
epoch 2000, loss 1.8053635358810425
epoch 1000, loss 0.9526720643043518
epoch 2000, loss 0.06419553607702255
epoch 1000, loss 0.5520960688591003
epoch 2000, loss 0.3331228196620941
epoch 3000, loss 0.3302273154258728
epoch 4000, loss 0.3297041654586792
epoch 1000, loss 4.516414165496826
epoch 2000, loss 4.255296230316162
epoch 3000, loss 0.04437210410833359
epoch 4000, loss 0.042907532304525375
epoch 1000, los

...

epoch 1000, loss 343.7348327636719
epoch 2000, loss 419.9408874511719
epoch 3000, loss 290.00653076171875
epoch 4000, loss 271.0689697265625
epoch 1000, loss 3782.02587890625
epoch 1000, loss 3603.69775390625
epoch 2000, loss 3218.1171875
epoch 3000, loss 3078.303466796875
epoch 1000, loss 2924.9189453125
epoch 2000, loss 2752.1298828125
epoch 1000, loss 3027.8828125
epoch 1000, loss 68.2757797241211
epoch 2000, loss 150.38937377929688
epoch 3000, loss 22.142688751220703
epoch 1000, loss 63.768096923828125
epoch 2000, loss 34.70985412597656
epoch 3000, loss 3.6241345405578613
epoch 4000, loss 0.29985764622688293
epoch 1000, loss 75.99095916748047
epoch 2000, loss 72.03807067871094
epoch 3000, loss 74.6034164428711
epoch 4000, loss 70.97029113769531
epoch 1000, loss 238.5487060546875
epoch 1000, loss 40.04981994628906
epoch 2000, loss 15.216239929199219
epoch 3000, loss 8.223626136779785
epoch 4000, loss 1.380252718925476
epoch 1000, loss 39.946372985839844
epoch 2000, loss 39.964351654

epoch 3000, loss 561.4152221679688
epoch 4000, loss 472.3698425292969
epoch 1000, loss 664.4426879882812
epoch 2000, loss 545.3485107421875
epoch 3000, loss 446.1298828125
epoch 4000, loss 364.0759582519531
epoch 1000, loss 154.0040283203125
epoch 2000, loss 148.21920776367188
epoch 3000, loss 139.34970092773438
epoch 4000, loss 134.49114990234375
epoch 1000, loss 98.63114929199219
epoch 2000, loss 86.85062408447266
epoch 1000, loss 2432.74169921875
epoch 2000, loss 2180.785888671875
epoch 3000, loss 1959.0360107421875
epoch 4000, loss 1759.0596923828125
epoch 1000, loss 392.0608825683594
epoch 2000, loss 373.779296875
epoch 3000, loss 358.3226318359375
epoch 4000, loss 344.6530456542969
epoch 1000, loss 539.439453125
epoch 2000, loss 526.246826171875
epoch 3000, loss 522.1149291992188
epoch 1000, loss 381.5697326660156
epoch 2000, loss 378.422607421875
epoch 3000, loss 359.1274719238281
epoch 4000, loss 356.0256042480469
epoch 1000, loss 3760.010498046875
epoch 2000, loss 3749.3713378

epoch 2000, loss 70397.859375
epoch 1000, loss -887858.625
epoch 2000, loss -1195738.0
epoch 3000, loss -1353666.375
epoch 1000, loss -15090394.0
epoch 1000, loss -10197909.0
epoch 1000, loss -1172474752.0
epoch 1000, loss -1016124032.0
epoch 1000, loss -634093504.0
epoch 1000, loss -354685452288.0
epoch 2000, loss -416459227136.0
epoch 3000, loss -584677326848.0
epoch 1000, loss 1.6060185432434082
epoch 2000, loss 1.5107831954956055
epoch 1000, loss 12.934724807739258
epoch 1000, loss 14.656583786010742
epoch 1000, loss 10.494189262390137
epoch 2000, loss 10.449769020080566
epoch 1000, loss 1.6640679836273193
epoch 2000, loss 1.463377833366394
epoch 1000, loss 8.024116516113281
epoch 2000, loss 7.991006851196289
epoch 3000, loss 7.988394737243652
epoch 1000, loss 1.427088975906372
epoch 2000, loss 1.398186206817627
epoch 3000, loss 1.3938425779342651
epoch 1000, loss 10.925333023071289
epoch 2000, loss 10.855567932128906
epoch 1000, loss 14.662635803222656
epoch 1000, loss 11.03697776

epoch 3000, loss 1.343453049659729
epoch 4000, loss 0.8719298839569092
epoch 1000, loss 17.254302978515625
epoch 2000, loss 11.288536071777344
epoch 3000, loss 11.297126770019531
epoch 4000, loss 11.27507209777832
epoch 1000, loss 5.287022113800049
epoch 2000, loss 5.064908027648926
epoch 3000, loss 5.0496745109558105
epoch 4000, loss 5.013909816741943
epoch 1000, loss 0.3446648120880127
epoch 2000, loss 16.717775344848633
epoch 3000, loss 9.52303695678711
epoch 1000, loss 1.9434702396392822
epoch 2000, loss 0.8612784743309021
epoch 3000, loss -0.1010550856590271
epoch 4000, loss -0.3504524230957031
epoch 1000, loss 2.6449074745178223
epoch 2000, loss 2.1838531494140625
epoch 3000, loss 2.1730329990386963
epoch 4000, loss 2.1284146308898926
epoch 1000, loss -8361.6396484375
epoch 2000, loss -21443.779296875
epoch 3000, loss -87919.4921875
epoch 4000, loss -117097.9765625
epoch 1000, loss -4205145.0
epoch 2000, loss -7470793.5
epoch 3000, loss -10117226.0
epoch 4000, loss -10183680.0
ep

epoch 2000, loss 14.048904418945312
epoch 3000, loss 13.091802597045898
epoch 4000, loss 12.620279312133789
epoch 1000, loss 14.654170036315918
epoch 2000, loss 7.820396423339844
epoch 1000, loss 11.93504524230957
epoch 2000, loss 10.348540306091309
epoch 3000, loss 9.410884857177734
epoch 1000, loss 16.760961532592773
epoch 1000, loss 6.095131874084473
epoch 2000, loss 5.792992115020752
epoch 1000, loss -68856.9296875
epoch 1000, loss -1032702.875
epoch 1000, loss -1510300.625
epoch 1000, loss -218867.78125
epoch 2000, loss -220864.734375
epoch 3000, loss -222665.09375
epoch 1000, loss -917350208.0
epoch 1000, loss -1.9251855101110367e+24
epoch 2000, loss -2.177853445992398e+24
epoch 3000, loss -2.2391551470834724e+24
epoch 4000, loss -2.2982884910547576e+24
epoch 1000, loss -8992106496.0
epoch 1000, loss -36132548608.0
epoch 1000, loss 0.9065800309181213
epoch 2000, loss 0.8993217945098877
epoch 3000, loss 0.8954002261161804
epoch 1000, loss 3.2848331928253174
epoch 2000, loss 1.9225